In [1]:
from midiutil import MIDIFile
import rebound
import numpy as np
import matplotlib.pyplot as plt
from subprocess import call
from itertools import repeat

In [11]:
class PlanetBeat():
    def __init__(self, filename, bpm, time_per_beat=None, dt=None, outer_midi_note=48, fps=30):
        try:
            call("rm -f ./tmp/*", shell=True)
        except:
            pass
        self.midi = MIDIFile(adjust_origin=True) # One track, defaults to format 1 (tempo track automatically created)
        self.filename = filename
        self.sim = rebound.Simulation.from_file(filename)
        self.sim.t = 0
        if time_per_beat:
            self.time_per_beat = time_per_beat
        else:
            self.time_per_beat = self.sim.particles[-1].P
        if not dt:
            self.sim.dt = self.sim.particles[1].P/100. # use small timestep to get transits right. This is not the bottleneck
        else:
            self.sim.dt = dt
        
        self.bpm = bpm
        self.fps = fps
        self.fig_ctr = 0
        self.time_elapsed = 0
        self.time_per_fig = 1./self.fps
        
        self.notes = self.calc_midi_notes(outer_midi_note)
        self.velocities = [100 for i in range(self.sim.N)]
        self.conjunction_notes = [12 for i in range(self.sim.N)] # C1
        self.conjunction_velocities = [100 for i in range(self.sim.N)]
        
        set_time_per_beat(self.sim, self.time_per_beat)
        self.change_tempo(bpm)
        self.fig_params = []
        self.conjunctions = []
    def calc_midi_notes(self, outer_midi_note):
        # 12 notes between octaves, freq = f0*2**(n/12). 
        # n = 12 log_2(freq/f0)
        # star, then planets from inside out
        ps = self.sim.particles
        midinotes = [0] # placeholder for star
        for p in ps[1:]:
            midinote = outer_midi_note+12*np.log(ps[-1].P/p.P)/np.log(2)
            midinotes.append(int(np.round(midinote)))
        return midinotes  
    def make_tuple(self, arg):
        N = self.sim.N
        if arg == True:
            return tuple(range(1,N))
        elif arg == False:
            return ()
        else:
            return tuple(arg)
    def integrate(self, tmax, color=True, duration=1, track=0, playtransits=True, playconjunctions=False, showplanets=True, showtransits=True, showconjunctions=True):
        playtransits = self.make_tuple(playtransits)
        playconjunctions = self.make_tuple(playconjunctions)
        showplanets = self.make_tuple(showplanets)
        showtransits = self.make_tuple(showtransits)
        showconjunctions = self.make_tuple(showconjunctions)
        
        N=self.sim.N
        ps = self.sim.particles
        yprev = np.zeros(N)
        sinthetaprev = np.zeros(N)
        while self.sim.t < tmax:
            self.sim.step()
            self.time_elapsed += self.sim.dt/self.bpm*60.
            for j in playtransits:
                if yprev[j] < 0 and ps[j].y > 0:
                    #print(self.sim.t, j)
                    #print(ps[j].index)
                    self.midi.addNote(track, ps[j].index, self.notes[j], self.sim.t, duration, self.velocities[j])
                yprev[j] = ps[j].y
            for j in playconjunctions:
                if j+1 in planets:
                    sintheta = np.sin(ps[j+1].theta-ps[j].theta)
                    if sinthetaprev[j] > 0 and sintheta < 0:
                        #print('conjunction', self.sim.t, j)
                        self.conjunctions.append((self.sim.t, j))
                        self.midi.addNote(track, N, self.conjunction_notes[j], self.sim.t, duration, self.conjunction_velocities[j]) # add to track above all planets
                    sinthetaprev[j] = sintheta
            if self.time_elapsed/self.time_per_fig > self.fig_ctr + 1:
                #print(self.time_elapsed*self.fps)
                self.fig_params.append([self.fig_ctr, self.sim.t, filename, self.time_per_beat, color, tuple(showplanets), tuple(showtransits), tuple(showconjunctions)])
                self.fig_ctr += 1
    def change_tempo(self, bpm):
        self.bpm = bpm
        self.midi.addTempo(0, self.sim.t, self.bpm) 
    def write_midi(self, filename):
        with open(filename, "wb") as f:
            self.midi.writeFile(f)

def set_time_per_beat(sim, time_per_beat): # makes sim.t run in units of the outer planet orbit = one beat
        ps = sim.particles
        sim.G = time_per_beat**2
        sim.dt /= time_per_beat
        for p in ps:
            p.vx *= time_per_beat
            p.vy *= time_per_beat
            p.vz *= time_per_beat
    
def write_png(params):
    fig_ctr, time, filename, time_per_beat, color, showplanets, showtransits, showconjunctions = params
    sim = rebound.Simulation.from_file(filename)
    sim.t=0
    set_time_per_beat(sim, time_per_beat)
    sim.integrate(time)
    ps = sim.particles
    
    lw=3
    scale=ps[1].a/5 # length scale for making dots bigger
    refsize=25*lw # this is what REBOUND uses for size of circles in call to plt.scatter

    fig = rebound.OrbitPlot(sim, figsize=(8,8), color=color, lw=lw, plotparticles=showplanets)
    ax = fig.axes[0]
    ax.axis('off')
    for i in showtransits:
        p = ps[i]
        if p.x > 0:
            ax.scatter(p.x, p.y, s=refsize*(1+5*np.exp(-np.abs(p.y)/scale)),color='k', marker='o')
    #if doconjunctions:
    #    nearby_conjunctions = [conjunction for conjunction in conjunctions if np.abs(conjunction[0]-time) < sim.particles[-1].P/10.]
    #    for conjunction in nearby_conjunctions:
    #        p = ps[conjunction[1]]
    #        ax.plot([0, p.x], [0,p.y], lw=5)
    fig.savefig('tmp/pngs/{0:0=5d}.png'.format(fig_ctr))
    plt.close(fig)  

In [79]:
filename = "trappist.bin"

pb = PlanetBeat(filename, bpm=30, outer_midi_note=48) # notes: http://subsynth.sourceforge.net/midinote2freq.html
planets = [6,7]

pb.integrate(7, duration=1, color=True)
    
midiname = "midi"
pb.write_midi("./tmp/"+midiname+".mid")

In [14]:
0.35/30*60*30

21.0

In [96]:
%%time
filename = "trappist.bin"

pb = PlanetBeat(filename, bpm=30, outer_midi_note=48, movie=True) # notes: http://subsynth.sourceforge.net/midinote2freq.html
planets = [-1]

N = 100
beats = np.linspace(0,10,N,endpoint=True)
tempos = np.linspace(30,100,N,endpoint=True)
for beat, tempo in zip(beats, tempos):
    pb.change_tempo(tempo)
    pb.integrate(beat, planets=planets, duration=1, color=True)
    
midiname = "midi"
pb.write_midi("./tmp/"+midiname+".mid")

0.028129343209743 -1
1.0283756944032767 -1
2.027794711972785 -1
3.0280410631661088 -1
4.027460080735617 -1
5.027706431928941 -1
6.027952783122265 -1
7.027371800691773 -1
8.027618151885155 -1
9.02703716945681 -1
CPU times: user 366 ms, sys: 499 ms, total: 866 ms
Wall time: 480 ms


In [16]:
20/30

0.6666666666666666

In [12]:
%%time
import warnings
warnings.filterwarnings("ignore")

call("rm -f tmp/pngs/*", shell=True)
pool = rebound.InterruptiblePool()
#for a in pb.fig_params:
#    a.append(pb.conjunctions)
res = pool.map(write_png, pb.fig_params)

CPU times: user 455 ms, sys: 151 ms, total: 605 ms
Wall time: 2min 34s


In [41]:
np.linspace(30,100,3, endpoint=True)

array([  30.,   65.,  100.])

In [42]:
list(range(6,1))

[]

In [47]:
np.linspace(28, 52, 7)

array([ 28.,  32.,  36.,  40.,  44.,  48.,  52.])

In [48]:
np.linspace(30, 100, 7)

array([  30.        ,   41.66666667,   53.33333333,   65.        ,
         76.66666667,   88.33333333,  100.        ])

In [10]:
#%%time
filename = "trappist.bin"

pb = PlanetBeat(filename, bpm=30, outer_midi_note=48) # notes: http://subsynth.sourceforge.net/midinote2freq.html
pb.conjunction_notes = [0, 33, 35, 20, 18, 14, 12]
pb.conjunction_velocities = [0, 43, 68, 100, 100, 124, 124]

planets = list(range(pb.sim.N))
for i in range(1,pb.sim.N):
    ps = planets[-i:]
    print(pb.sim.t, ps)
    pb.integrate(tmax=pb.sim.t+4, playtransits=ps, playconjunctions=False, showplanets=ps, showtransits=ps, showconjunctions=False, duration=1, color='red')
    
conjunctionbars = 6
barbpms = np.linspace(30,100,conjunctionbars+1)
N = 10

for j, i in enumerate(range(2,pb.sim.N)):
    ps = planets[-i:]
    print(pb.sim.t, ps, pb.bpm)
    times = np.linspace(pb.sim.t,pb.sim.t+4,N,endpoint=True)
    bpms = np.linspace(pb.bpm,barbpms[j+1],N,endpoint=True)
    for time, bpm in zip(times, bpms):
        pb.change_tempo(bpm)
        pb.integrate(time, playtransits=True, playconjunctions=ps, showplanets=ps, showtransits=ps, showconjunctions=ps, duration=1, color='red')

for i in range(4):
    print(pb.sim.t, ps, pb.bpm)
    pb.integrate(tmax=pb.sim.t+4, playtransits=True, playconjunctions=True, showplanets=True, showtransits=True, showconjunctions=True, duration=1, color='red')

pb.change_tempo(30)    
print(pb.sim.t, ps, pb.bpm)

midiname = "midi"
pb.write_midi("./tmp/"+midiname+".mid")

0.0 [7]
4.000158071149695 [6, 7]
8.000316142299175 [5, 6, 7]
12.000474213457244 [4, 5, 6, 7]
16.00063228461531 [3, 4, 5, 6, 7]
20.00079035577338 [2, 3, 4, 5, 6, 7]
24.000948426931448 [1, 2, 3, 4, 5, 6, 7]
28.001106498089516 [6, 7] 30
32.00126456924758 [5, 6, 7] 41.6666666667
36.00142264040565 [4, 5, 6, 7] 53.3333333333
40.00158071156372 [3, 4, 5, 6, 7] 65.0
44.00173878272179 [2, 3, 4, 5, 6, 7] 76.6666666667
48.001896853879856 [1, 2, 3, 4, 5, 6, 7] 88.3333333333
52.002054925037925 [1, 2, 3, 4, 5, 6, 7] 100.0
56.00221299619599 [1, 2, 3, 4, 5, 6, 7] 100.0
60.00237106735406 [1, 2, 3, 4, 5, 6, 7] 100.0
64.00252913851213 [1, 2, 3, 4, 5, 6, 7] 100.0
68.0026872096702 [1, 2, 3, 4, 5, 6, 7] 30


In [86]:
barbpms

array([  30.        ,   41.66666667,   53.33333333,   65.        ,
         76.66666667,   88.33333333,  100.        ])

In [13]:
call("timidity -Ow ./tmp/{0}.mid -o ./tmp/{0}.wav --preserve-silence".format(midiname), shell=True)
call("ffmpeg -t {0} -i ./tmp/{1}.wav ./tmp/{1}cut.wav".format(pb.time_elapsed, midiname), shell=True)
moviename = "test2.mp4"
fps = 30
try:
    call("rm -f {0}".format(moviename), shell=True)
except:
    pass
call("ffmpeg -r {0} -i tmp/pngs/%05d.png -i tmp/{1}cut.wav -c:v libx264 -pix_fmt yuv420p -c:a libvo_aacenc -b:a 192k -shortest {2}".format(fps, midiname, moviename), shell=True)

0

In [14]:
call("open "+moviename, shell=True)

0

In [79]:
call("open ./tmp/midi.wav", shell=True)

0

In [70]:
midiname

'midi'

In [75]:
"./tmp/{0}cut.wav".format(midiname)

'./tmp/midicut.wav'